In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import numpy as np
pd.set_option('display.max_columns', None)
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel


### File reading and cleaning

In [48]:
customer_churn = pd.read_csv("file:///Users/genevauy/Desktop/DATA_Customer-Churn.csv")
customer_churn

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Male,0,Yes,Yes,24,Yes,Yes,No,Yes,Yes,Yes,Yes,One year,84.80,1990.5,No
7039,Female,0,Yes,Yes,72,Yes,No,Yes,Yes,No,Yes,Yes,One year,103.20,7362.9,No
7040,Female,0,Yes,Yes,11,No,Yes,No,No,No,No,No,Month-to-month,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,No,No,No,No,No,No,Month-to-month,74.40,306.6,Yes


In [53]:
customer_churn['TotalCharges'] = pd.to_numeric(customer_churn['TotalCharges'], downcast='float', errors='coerce')
customer_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   OnlineSecurity    7043 non-null   object 
 7   OnlineBackup      7043 non-null   object 
 8   DeviceProtection  7043 non-null   object 
 9   TechSupport       7043 non-null   object 
 10  StreamingTV       7043 non-null   object 
 11  StreamingMovies   7043 non-null   object 
 12  Contract          7043 non-null   object 
 13  MonthlyCharges    7043 non-null   float64
 14  TotalCharges      7043 non-null   float32
 15  Churn             7043 non-null   object 
dtypes: float32(1), float64(1), int64(2), objec

In [54]:
mean_totalcharges = customer_churn['TotalCharges'].mean()
customer_churn['TotalCharges'].fillna(mean_totalcharges, inplace=True)


In [55]:
selected_columns = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges', 'Churn']
df = customer_churn[selected_columns]


In [56]:
df = df.rename(columns=lambda x: x.lower())


In [57]:
df["churn"] = df["churn"].apply(lambda x: x.replace('No', '0') if isinstance(x, str) else x)
df["churn"] = df["churn"].apply(lambda x: x.replace('Yes', '1') if isinstance(x, str) else x)

In [58]:
df['churn'] = df['churn'].astype(int)

### Test-Train Split

In [59]:
# Splitting the data into train and test sets
X = df[["tenure", "seniorcitizen", "monthlycharges", "totalcharges"]]
y = df['churn']

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)


### Balancing using SMOTE

In [61]:
# Balancing the data using SMOTE
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

### Scaling Using Minmax Scaler

In [62]:
# Scaling the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Decision Tree Classifier

In [63]:
# Decision Tree Classification
model = DecisionTreeClassifier(max_depth=5)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [64]:
print("The (mean) accuracy on the test set is %.2f" % model.score(X_test, y_test))
print("The (mean) accuracy on the train data is %.2f" % model.score(X_train, y_train))
print("X has %d rows and %d columns" % (X.shape[0], X.shape[1]))
print("y has %d rows" % y.shape[0])

The (mean) accuracy on the test set is 0.73
The (mean) accuracy on the train data is 0.76
X has 7043 rows and 4 columns
y has 7043 rows


### Cross Validation

In [65]:
scores = cross_val_score(model, X_train, y_train, cv=5)
print("Cross-validation scores:", scores)
print("Mean accuracy with a standard deviation of %.2f" % scores.mean())


Cross-validation scores: [0.7291918  0.73325287 0.74652987 0.73989137 0.74411587]
Mean accuracy with a standard deviation of 0.74


In [66]:
y_pred = cross_val_predict(model, X_test, y_test, cv=5)
y_pred

array([1, 1, 0, ..., 0, 0, 0])

In [67]:
model1 = DecisionTreeClassifier(max_depth=5)
model2 = LogisticRegression()
model3 = KNeighborsClassifier()

In [68]:
model_pipeline = [model1, model2, model3]
model_names = ['Classification Tree', 'Logistic Regression', 'KNN']
scores = {}
for model, name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train, y_train, cv=5))
    scores[name] = mean_score

In [69]:
print("Comparing the 3 regression scores:")
pd.DataFrame([scores], index=["score"])

Comparing the 3 regression scores:


,Classification Tree,Logistic Regression,KNN
score,0.738596,0.726769,0.749942


### Random Forest

In [70]:
# Random Forest
rfc_ops = {
    "max_depth": 6,
    "min_samples_leaf": 20,
    "n_estimators": 100,
    "bootstrap": True,
    "oob_score": True,
    "random_state": 42
}

clf = RandomForestClassifier(**rfc_ops)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=20, oob_score=True,
                       random_state=42)

In [71]:
print("Train prediction accuracy score: %.2f" % clf.score(X_train, y_train))
print("Test prediction accuracy score: %.2f" % clf.score(X_test, y_test))

Train prediction accuracy score: 0.77
Test prediction accuracy score: 0.73


In [72]:
score_ds = accuracy_score(y_test, clf.predict(X_test))
print("Test prediction accuracy score: %.2f" % score_ds)

Test prediction accuracy score: 0.73


### Grid Search

In [73]:
# Grid Search
param_grid = {
    'n_estimators': [50, 100],
    'min_samples_split': [80],
    'min_samples_leaf': [50],
    'max_features': ['sqrt'],
    'max_depth': [3, 5],
}

In [74]:
clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(clf, param_grid, cv=5, return_train_score=True, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print("Best parameters:", best_params)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [3, 5], 'max_features': ['sqrt'],
                         'min_samples_leaf': [50], 'min_samples_split': [80],
                         'n_estimators': [50, 100]},
             return_train_score=True)

Best parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 50, 'min_samples_split': 80, 'n_estimators': 100}


In [75]:
cv_results_df = pd.DataFrame(grid_search.cv_results_)
cv_results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.238805,0.017925,0.007164,0.002770,3,sqrt,50,80,50,"{'max_depth': 3, 'max_features': 'sqrt', 'min_...",0.726779,0.728425,0.739891,0.712734,0.743512,0.730268,0.010872,4,0.735969,0.745361,0.734802,0.730427,0.744456,0.738203,0.005786
1,0.483197,0.033982,0.021058,0.002137,3,sqrt,50,80,100,"{'max_depth': 3, 'max_features': 'sqrt', 'min_...",0.729192,0.728425,0.750754,0.727821,0.743512,0.735941,0.009431,3,0.737477,0.749133,0.739629,0.740987,0.742495,0.741944,0.003955
2,0.301796,0.017885,0.010852,0.004458,5,sqrt,50,80,50,"{'max_depth': 5, 'max_features': 'sqrt', 'min_...",0.735223,0.742909,0.759203,0.732649,0.753168,0.744631,0.010201,2,0.762070,0.759541,0.755770,0.759240,0.755318,0.758388,0.002526
3,0.473897,0.046862,0.015309,0.006369,5,sqrt,50,80,100,"{'max_depth': 5, 'max_features': 'sqrt', 'min_...",0.732207,0.742305,0.762221,0.735063,0.756789,0.745717,0.011854,1,0.763277,0.758033,0.758335,0.763614,0.755468,0.759745,0.003183


In [76]:
clf = RandomForestClassifier(random_state=42, **best_params)
cross_val_scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Mean cross-validation accuracy score: %.2f" % np.mean(cross_val_scores))

Mean cross-validation accuracy score: 0.75


### Feature Importance

In [77]:
# Feature Importance
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, min_samples_leaf=50, min_samples_split=80,
                       random_state=42)

In [80]:
feature_importance = clf.feature_importances_
feature_names = X.columns

In [81]:
df_feature_importance = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
df_feature_importance.sort_values(by='Importance', ascending=False, inplace=True)
df_feature_importance

,Feature,Importance
0,tenure,0.453929
2,monthlycharges,0.350086
3,totalcharges,0.195185
1,seniorcitizen,0.000799
